In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

In [2]:
# data = pd.read_csv('wheat-seeds.csv', names = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'y'], sep = ',', dtype = np.float)
# train = data.sample(150)
# test = data.loc[list(filter(lambda x: x not in train.index, data.index))]
# testTarget = torch.tensor(test['y'].values.reshape((56, 1)), dtype = torch.float32)
# test = torch.tensor(test[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7']].values, dtype = torch.float32)
# testTarget[testTarget != 1] = 0
# X = torch.tensor(train[['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7']].values, dtype = torch.float32)
# Y = torch.tensor(train[['y']].values, dtype = torch.float32)
# Y[Y != 1.] = 0

In [3]:
data = pd.read_csv('dados.csv', names = ['x1', 'x2', 'x3', 'y'], dtype = np.float)
data['y'][data['y'] == -1] = 0
X = torch.tensor(data[['x1', 'x2', 'x3']].values, dtype = torch.float32)
Y = torch.tensor(data[['y']].values, dtype = torch.float32)

For some reason, it's not working when the datatype is set to float64.

In [7]:
class LinearNet(nn.Module):
    def __init__(self):
        super(LinearNet, self).__init__()
        self.fc1 = nn.Linear(3, 1)
        self.loss = nn.BCELoss()
        
    def forward(self, X):
        X = self.fc1(X)
        X = torch.sigmoid(X)
        return X
    
    def fit(self, X, target, epochs = 10, learningRate = 0.01, cuda = False):
        index = np.arange(len(X))
        optimizer = torch.optim.SGD(self.parameters(), lr = learningRate)
        LossVec = []
        for epoch in range(epochs):
            np.random.shuffle(index)
            for i in index:
                optimizer.zero_grad()
                
                output = self.forward(X[i])
                loss = self.loss(output, target[i]); 
                loss.backward()
                
                optimizer.step()   
            LossVec.append(loss.item())
                
        return LossVec

In [8]:
net = LinearNet()

In [14]:
loss = net.fit(X, Y, epochs = 10, learningRate = 0.01/2)
fig, ax = plt.subplots(figsize = (10, 5))
ax.plot(np.arange(len(loss)), loss,  color = 'k')

W = list(net.parameters())[0][0]
a = W[0].item()
b = W[1].item()
c = W[2].item()
bias = list(net.parameters())[1].item()

fig1 = plt.figure()
ax1 = Axes3D(fig1)
point = X

xx, yy = np.meshgrid(range(-3, 5), range(-5, 5))
z = (-a*xx - b*yy - bias)*1. / c
surface1 = [xx, yy, z]

C1 = data.where(data["y"] == 1).dropna().index
C2 = data.where(data["y"] == 0).dropna().index

ax1.scatter(data["x1"][C1], data["x2"][C1], data["x3"][C1], color = "r", s=50)
ax1.scatter(data["x1"][C2], data["x2"][C2], data["x3"][C2], color = "b", s=50)

ax1.plot_surface(xx, yy, z, color = 'grey')

In [167]:
loss[-5:]

[0.10117460787296295,
 0.07752209901809692,
 0.19316145777702332,
 0.06095417961478233,
 0.0030102096498012543]

In [168]:
net.loss(net(test), testTarget)

tensor(0.3707, grad_fn=<BinaryCrossEntropyBackward>)